_Author: Dmitry Bagaev_

In [ ]:
using DrWatson

In [ ]:
@quickactivate "RxInferThesisExperiments"

In [ ]:
using RxInferThesisExperiments, Turing, StaticArrays, Plots, PGFPlotsX, LaTeXStrings, ReverseDiff
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames, Logging

In [ ]:
const bfolder = datadir("nlds", "turing", "nuts")

In [ ]:
# Pregenerate paths for benchmark data
mkpath(bfolder);

In [ ]:
# Create default environment with default parameters
# We reduce the time-step and noise a little bit to test for very large models
# Otherwise the systems is too unstable and leads to NaNs
const environment = DoublePendulum(Δt = 0.0001, γ = 0.1)

In [ ]:
# Define state-transition function, uses RK4 method internally, see the `src/` folder
f(state) = state_transition(environment)(state)

In [ ]:
# Include the model specification
include(srcdir("models", "turing", "doublependulum.jl"));

In [ ]:
function run_benchmark(params)
    return with_logger(NullLogger()) do
        @unpack T, nsamples, seed = params

        states, observations = rand(StableRNG(seed), environment, T);
        model    = double_pendulum(observations, T)
        method   = NUTS()
        result   = sample_inference(model, method = method, nsamples = nsamples, rng = StableRNG(seed))
        e_states = extract_posteriors(T, result)
        amse     = compute_amse(states, e_states)

        benchmark_modelcreation = @benchmark double_pendulum($observations, $T)

        benchmark_inference = @benchmark sample_inference(model, method = $method; nsamples = $nsamples, rng = StableRNG($seed)) setup=begin
            states, observations = rand(StableRNG($seed), environment, $T);
            model = double_pendulum(observations, $T)
        end

        emse = compute_emse(seed) do _seed
            local states, observations = rand(StableRNG(_seed), environment, T; random_start = true);
            local model    = double_pendulum(observations, T)
            local method   = NUTS()
            local result   = sample_inference(model, method = method, nsamples = nsamples, rng = StableRNG(_seed))
            local e_states = extract_posteriors(T, result)
            return compute_amse(states, e_states)
        end

        output = @strdict T nsamples seed states e_states observations amse emse benchmark_modelcreation benchmark_inference

        return output
    end
end

In [ ]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"           => [ 10, 20, 30, 50, 100, 200 ],
    "nsamples"    => [ 50, 100 ],
    "seed"        => [ 42 ]
));

In [ ]:
# Disable turing's show progress as it hurts performance (a bit)
Turing.setprogress!(false)

In [ ]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    result, _ = produce_or_load(run_benchmark, bfolder, params; tag = false, force = false)
    return result
end;

In [ ]:
sort(prepare_benchmarks_table(bfolder), [ :T, :nsamples ])

# Versions

In [ ]:
versioninfo()

In [ ]:
] status